In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,TimeDistributed,GRU,SimpleRNN
from keras.preprocessing.sequence import TimeseriesGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
label_cols=[]

In [3]:
articles_df=pd.read_csv(r'articles.csv')
customers_df=pd.read_csv(r'customers.csv')
transcations_df=pd.read_csv(r'transactions_train.csv')

In [4]:
articles_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [6]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [7]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [8]:
transcations_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [9]:
def missing_data(func_df):
    total=func_df.isnull().sum().sort_values(ascending=False)
    percent=(func_df.isnull().sum()/func_df.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total,percent],axis=1,keys=['Total','Percent'])

In [10]:
missing_data(articles_df)

,Total,Percent
detail_desc,416,0.394156
perceived_colour_master_name,0,0.000000
garment_group_name,0,0.000000
garment_group_no,0,0.000000
section_name,0,0.000000
section_no,0,0.000000
index_group_name,0,0.000000
index_group_no,0,0.000000
index_name,0,0.000000
index_code,0,0.000000


In [11]:
missing_data(customers_df)

,Total,Percent
Active,907576,66.150819
FN,895050,65.237831
fashion_news_frequency,16009,1.166854
age,15861,1.156066
club_member_status,6062,0.441843
customer_id,0,0.000000
postal_code,0,0.000000


In [12]:
missing_data(transcations_df)

,Total,Percent
t_dat,0,0.0
customer_id,0,0.0
article_id,0,0.0
price,0,0.0
sales_channel_id,0,0.0


In [13]:
def unique_values(func_df):
    total=func_df.count()
    tt=pd.DataFrame(total)
    tt.columns=['Total']
    uniques=[]
    for col in func_df.columns:
        unique=func_df[col].nunique()
        uniques.append(unique)
    tt['Uniques']=uniques
    return tt

In [14]:
unique_values(articles_df)

,Total,Uniques
article_id,105542,105542
product_code,105542,47224
prod_name,105542,45875
product_type_no,105542,132
product_type_name,105542,131
product_group_name,105542,19
graphical_appearance_no,105542,30
graphical_appearance_name,105542,30
colour_group_code,105542,50
colour_group_name,105542,50


In [15]:
unique_values(customers_df)

,Total,Uniques
customer_id,1371980,1371980
FN,476930,1
Active,464404,1
club_member_status,1365918,3
fashion_news_frequency,1355971,4
age,1356119,84
postal_code,1371980,352899


In [16]:
# this block of code is used to map prod_name and product_code
index_df = pd.DataFrame(columns = ['product_no','product_name'])
for index,row in articles_df.iterrows():
    if row['product_code'] not in index_df.product_no.unique():
        index_df=index_df.append({'product_no':row['product_code'],'product_name':row['prod_name']},ignore_index = True)

In [17]:
index_df.head()

,product_no,product_name
0,108775,Strap top
1,110065,OP T-shirt (Idro)
2,111565,20 den 1p Stockings
3,111586,Shape Up 30 den 1p Tights
4,111593,Support 40 den 1p Tights


In [18]:
index_df.shape

(47224, 2)

In [19]:
index_df.sort_values('product_no')

,product_no,product_name
0,108775,Strap top
1,110065,OP T-shirt (Idro)
2,111565,20 den 1p Stockings
3,111586,Shape Up 30 den 1p Tights
4,111593,Support 40 den 1p Tights
...,...,...
47219,953450,5pk regular Placement1
47220,953763,SPORT Malaga tank
47221,956217,Cartwheel dress
47222,957375,CLAIRE HAIR CLAW


In [20]:
index_df.to_csv('product_label_dataset',encoding='utf-8',index=False)

In [21]:
unique_values(transcations_df)

,Total,Uniques
t_dat,31788324,734
customer_id,31788324,1362281
article_id,31788324,104547
price,31788324,9857
sales_channel_id,31788324,2


### Here i am dropping this columns because it has the column which represents in the number format in simple words to reduce multicollinearity
### Eg product_name column has product_name which is same 

In [22]:
cols=['prod_name','product_type_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name',
      'department_name','index_name','index_group_name','section_name','garment_group_name']
articles_df.drop(columns=cols,axis=1,inplace=True)

In [23]:
articles_df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no,detail_desc
0,108775015,108775,253,Garment Upper body,1010016,9,4,5,Black,1676,A,1,16,1002,Jersey top with narrow shoulder straps.
1,108775044,108775,253,Garment Upper body,1010016,10,3,9,White,1676,A,1,16,1002,Jersey top with narrow shoulder straps.
2,108775051,108775,253,Garment Upper body,1010017,11,1,9,White,1676,A,1,16,1002,Jersey top with narrow shoulder straps.
3,110065001,110065,306,Underwear,1010016,9,4,5,Black,1339,B,1,61,1017,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,306,Underwear,1010016,10,3,9,White,1339,B,1,61,1017,"Microfibre T-shirt bra with underwired, moulde..."


In [24]:
label_encoder = preprocessing.LabelEncoder()
cols=['product_group_name','perceived_colour_master_name','index_code']
for col in cols:
    label_cols.append(col)
    articles_df[col]= label_encoder.fit_transform(articles_df[col]) 

### I will be dropping detail_desc column 

In [25]:
articles_df.drop(['detail_desc'],axis=1,inplace=True)

In [26]:
articles_df

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no
0,108775015,108775,253,7,1010016,9,4,5,1,1676,0,1,16,1002
1,108775044,108775,253,7,1010016,10,3,9,16,1676,0,1,16,1002
2,108775051,108775,253,7,1010017,11,1,9,16,1676,0,1,16,1002
3,110065001,110065,306,16,1010016,9,4,5,1,1339,1,1,61,1017
4,110065002,110065,306,16,1010016,10,3,9,16,1339,1,1,61,1017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,302,13,1010014,9,4,5,1,7188,4,3,26,1021
105538,953763001,953763,253,7,1010016,9,4,5,1,1919,0,1,2,1005
105539,956217002,956217,265,5,1010016,9,4,5,1,1641,0,1,18,1005
105540,957375001,957375,72,0,1010016,9,4,5,1,3946,3,2,52,1019


In [27]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [28]:
customers_df.shape

(1371980, 7)

In [29]:
customers_df['fashion_news_frequency'].value_counts()

NONE         877711
Regularly    477416
Monthly         842
None              2
Name: fashion_news_frequency, dtype: int64

In [30]:
print("The precentage of null value in FN column is {}%".format(round(((customers_df['FN'].isna().sum()/customers_df.shape[0])*100),2)))
print("The precentage of null value in Active column is {}%".format(round(((customers_df['Active'].isna().sum()/customers_df.shape[0])*100),2)))

The precentage of null value in FN column is 65.24%
The precentage of null value in Active column is 66.15%


### I am dropping the FN and Active column because it has more than 65 % null values 

In [31]:
customers_df.drop(['FN','Active'],axis=1,inplace=True)

In [32]:
customers_df['club_member_status'].isna().sum()

6062

In [33]:
customers_df['club_member_status'].fillna('ACTIVE',axis=0,inplace=True)

In [34]:
customers_df['club_member_status'].isna().sum()

0

In [35]:
customers_df['fashion_news_frequency'].replace(to_replace='NONE',value='None',inplace=True)

In [36]:
customers_df['fashion_news_frequency'].value_counts()

None         877713
Regularly    477416
Monthly         842
Name: fashion_news_frequency, dtype: int64

In [37]:
customers_df['postal_code'].nunique()

352899

### Since we has so many unique values in postal_code so  i will be dropping this column

In [38]:
customers_df.drop(['postal_code'],axis=1,inplace=True)

In [39]:
customers_df.dropna(axis=0,inplace=True)

In [40]:
label_encoder = preprocessing.LabelEncoder()
cols=['club_member_status','fashion_news_frequency']
for col in cols:
    label_cols.append(col)
    customers_df[col]= label_encoder.fit_transform(customers_df[col]) 

In [41]:
customers_df.head()

,customer_id,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,1,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,1,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,1,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,1,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0,2,52.0


In [42]:
transcations_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### There is no null values in transcations 

In [43]:
transcations_df['t_dat']=pd.to_datetime(transcations_df["t_dat"])

In [44]:
transcations_df.tail()

,t_dat,customer_id,article_id,price,sales_channel_id
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1
31788323,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003,0.033881,2


In [45]:
transcations_df.shape

(31788324, 5)

### Here i have selected only the transcations after the year 2020

In [46]:
transcations_df=transcations_df[transcations_df['t_dat']>'2020-01-01'].copy()
transcations_df.shape

(10967372, 5)

In [47]:
transcations_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
20820952,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565002,0.030492,2
20820953,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565001,0.030492,2
20820954,2020-01-02,00067622de3151a7219b4ed9922def50b51601fbe41418...,801865004,0.018627,1
20820955,2020-01-02,0010f56acce349e6e82bfef13ee39232a8bc0db0801ca4...,578752001,0.020322,2
20820956,2020-01-02,0010f56acce349e6e82bfef13ee39232a8bc0db0801ca4...,578752001,0.020339,2


In [48]:
inner_merged=pd.merge(transcations_df,articles_df)#inner merge 

In [49]:
inner_merged.shape

(10967372, 18)

In [50]:
inner_merged.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no
0,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565002,0.030492,2,797565,306,16,1010016,42,7,18,13,1338,1,1,61,1017
1,2020-01-02,0a472ecdcdf4c940007edc47e9274a525ef2f0350f2a57...,797565002,0.030492,2,797565,306,16,1010016,42,7,18,13,1338,1,1,61,1017
2,2020-01-02,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,797565002,0.029695,2,797565,306,16,1010016,42,7,18,13,1338,1,1,61,1017
3,2020-01-02,6593c2dcf880bf65dc68a3c3e6b29646bf95b0e93fae09...,797565002,0.030492,2,797565,306,16,1010016,42,7,18,13,1338,1,1,61,1017
4,2020-01-02,a4eccd4d68777d7e6076ef26e888ac6dbfa34d371fae6f...,797565002,0.030492,2,797565,306,16,1010016,42,7,18,13,1338,1,1,61,1017


In [51]:
final_merged=pd.merge(inner_merged,customers_df)

In [52]:
final_merged.shape

(10906163, 21)

In [53]:
final_merged.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,...,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no,club_member_status,fashion_news_frequency,age
0,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565002,0.030492,2,797565,306,16,1010016,42,...,18,13,1338,1,1,61,1017,0,2,25.0
1,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565001,0.030492,2,797565,306,16,1010016,9,...,5,1,1338,1,1,61,1017,0,2,25.0
2,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,751471001,0.033881,2,751471,272,6,1010016,9,...,5,1,1722,0,1,15,1009,0,2,25.0
3,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,781613006,0.050831,2,781613,264,7,1010016,9,...,5,1,1222,0,1,15,1008,0,2,25.0
4,2020-04-24,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,855823003,0.025407,2,855823,265,5,1010016,9,...,5,1,1344,3,2,53,1013,0,2,25.0


### We dont need article_id and customer_id anymore so we will be dropping that too

In [54]:
# final_merged.drop(['customer_id','article_id'],axis=1,inplace=True)

In [55]:
int_cols=[]
total_columms=final_merged.columns
for col in total_columms:
    if col not in label_cols:
        int_cols.append(col)

In [56]:
final_merged[int_cols]

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,product_type_no,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_group_no,section_no,garment_group_no,age
0,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565002,0.030492,2,797565,306,1010016,42,7,18,1338,1,61,1017,25.0
1,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565001,0.030492,2,797565,306,1010016,9,4,5,1338,1,61,1017,25.0
2,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,751471001,0.033881,2,751471,272,1010016,9,4,5,1722,1,15,1009,25.0
3,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,781613006,0.050831,2,781613,264,1010016,9,4,5,1222,1,15,1008,25.0
4,2020-04-24,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,855823003,0.025407,2,855823,265,1010016,9,4,5,1344,2,53,1013,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10906158,2020-09-22,10d455608b1d73268f0a9ff1a392ccd681e665ed471f6d...,932182001,0.033881,2,932182,256,1010016,9,4,5,1636,1,15,1005,26.0
10906159,2020-09-22,b834a2cb440d2298493d254196abb73e101d8b35b155c6...,870982002,0.061000,2,870982,261,1010016,12,1,11,1244,2,53,1007,62.0
10906160,2020-09-22,83b8a97fd3decf8e21d05a9acaea822eaa569c957ea232...,893427001,0.084729,2,893427,267,1010018,9,4,5,1722,1,15,1009,22.0
10906161,2020-09-22,83b8a97fd3decf8e21d05a9acaea822eaa569c957ea232...,893427001,0.084729,2,893427,267,1010018,9,4,5,1722,1,15,1009,22.0


In [57]:
final_merged.columns.intersection(label_cols)

Index(['product_group_name', 'perceived_colour_master_name', 'index_code',
       'club_member_status', 'fashion_news_frequency'],
      dtype='object')

In [58]:
# final_merged.to_csv('cleaned_h_m_dataset',encoding='utf-8',index=False)

In [59]:
final_merged=pd.read_csv(r'cleaned_h_m_dataset')

In [60]:
final_merged.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,...,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no,club_member_status,fashion_news_frequency,age
0,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565002,0.030492,2,797565,306,16,1010016,42,...,18,13,1338,1,1,61,1017,0,2,25.0
1,2020-01-02,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,797565001,0.030492,2,797565,306,16,1010016,9,...,5,1,1338,1,1,61,1017,0,2,25.0
2,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,751471001,0.033881,2,751471,272,6,1010016,9,...,5,1,1722,0,1,15,1009,0,2,25.0
3,2020-08-14,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,781613006,0.050831,2,781613,264,7,1010016,9,...,5,1,1222,0,1,15,1008,0,2,25.0
4,2020-04-24,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,855823003,0.025407,2,855823,265,5,1010016,9,...,5,1,1344,3,2,53,1013,0,2,25.0


In [61]:
final_merged.t_dat.max()

'2020-09-22'

In [62]:
df_train=final_merged[(final_merged['t_dat']>'2020-08-28') & (final_merged['t_dat']<'2020-09-20')].copy()
df_test=final_merged[final_merged['t_dat']>='2020-09-20'].copy()

In [63]:
df_train.t_dat.max()

'2020-09-19'

In [64]:
df_train.drop(['t_dat'],axis=1,inplace=True)
df_test.drop(['t_dat'],axis=1,inplace=True)

In [38]:
cols=df_train.columns

In [65]:
df_train.head()

,customer_id,article_id,price,sales_channel_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,perceived_colour_master_name,department_no,index_code,index_group_no,section_no,garment_group_no,club_member_status,fashion_news_frequency,age
28,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,794575002,0.033881,2,794575,308,7,1010016,11,1,9,16,1643,3,2,51,1002,0,2,26.0
65,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,877711001,0.059305,2,877711,92,12,1010016,9,4,5,1,3528,2,1,64,1020,0,2,26.0
72,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,878190004,0.067780,2,878190,262,7,1010016,9,4,5,1,1201,0,1,19,1007,0,2,26.0
75,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,909014001,0.118627,2,909014,262,7,1010016,9,4,5,1,1201,0,1,19,1007,0,2,26.0
76,470dac37792d1e124865d249d1e304f136c11ea46d2ec9...,898694001,0.050831,2,898694,264,7,1010010,12,1,11,0,1222,0,1,15,1008,0,2,26.0


In [ ]:
# df_product_code_train=df_train.copy()
# scaler=MinMaxScaler()
# df_train=pd.DataFrame(scaler.fit_transform(df_train['']),columns=cols)
# df_test=pd.DataFrame(scaler.transform(df_test),columns=cols)
# scaler_pred=MinMaxScaler()
# df_product_code=pd.DataFrame(scaler_pred.fit_transform(df_product_code_train.product_code.values.reshape(-1,1)))

In [ ]:
# y_train=df_train['product_code'].tolist()
# y_test=df_test['product_code'].tolist()
# df_train.drop(['product_code'],axis=1,inplace=True)
# df_test.drop(['product_code'],axis=1,inplace=True)
# x_train=df_train.to_numpy().tolist()
# x_test=df_test.to_numpy().tolist()

In [94]:
train_series=df_train['product_code'].values.reshape((-1, 1))
test_series=df_test['product_code'].values.reshape((-1, 1))
scaler=MinMaxScaler()
train_series=scaler.fit_transform(train_series)
test_series=scaler.transform(test_series)
scaler_pred=MinMaxScaler()
train_generator = TimeseriesGenerator(data=train_series, targets=train_series,length=10)
test_generator = TimeseriesGenerator(data=test_series, targets=test_series,length=10)

In [95]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
    keras.layers.LSTM(512, input_shape = (10, 1), return_sequences=True),
    keras.layers.LSTM(512, return_sequences=True),
    keras.layers.LSTM(512, return_sequences=True),
    keras.layers.LSTM(512, return_sequences=True),
    keras.layers.LSTM(512, return_sequences=True),
    keras.layers.Dense(1)
])

In [96]:
adam_optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=adam_optimizer, loss = 'mean_squared_error')
history = model.fit(train_generator, validation_data = test_generator, epochs = 10)

Epoch 1/10
1054/6346 [===>..........................] - ETA: 6:42 - loss: 0.0396

KeyboardInterrupt: 

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
from sklearn.metrics import mean_squared_error
# print(mean_squared_error(predictions,y_test))


recommdation like apriori lstm